In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# 예시 설정
examples = [
    {"movie": "탑건", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "emojis": "👨‍👨‍👦🔫🍝"},
    {"movie": "해리포터", "emojis": "🧙‍♂️⚡🦉"},
    {"movie": "타이타닉", "emojis": "🚢🌊💑"},
]

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["movie", "emojis"],
        template="Human: {movie}\nAI: {emojis}"
    ),
    max_length=50,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{movie}"), ("ai", "{emojis}")]
    ),
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant that responds with exactly three emojis representing movies. If you ask the movie you asked before, please answer only the title of the movie and the 3 emojis you expressed."),
        MessagesPlaceholder(variable_name="history"),
        few_shot_prompt,
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(movie_name):
    result = chain.invoke({"question": movie_name})
    memory.save_context(
        {"input": movie_name},
        {"output": result.content},
    )
    print(result.content)

In [2]:
invoke_chain("매트릭스")

💊💻🕶️


In [3]:
invoke_chain("인셉션")

🧠🌌🔑


In [4]:
invoke_chain("내가 처음 물어본 영화는 무엇이었지?")

매트릭스 💊💻🕶️
